# Network Analysis

In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os

SOC_URL = "./actorMovies.csv"
df = pd.read_csv(SOC_URL, sep=";")


for i in range(df["Movies"].count()):
    df["Movies"][i] = df["Movies"][i].split("|")

df.head(25)

total = 0
for i in range(df["Movies"].count()):
    total += len(df["Movies"][i])
print(total)
df.head()

10247


,Actor,Movies
0,Karen Allen,"[Malcolm X, National Lampoon's Animal House, S..."
1,Taye Diggs,"[House on Haunted Hill, Go, Basic, Chicago, Eq..."
2,Michael Murphy,"[Salvador, Cloak & Dagger, Nashville, Salvador..."
3,Les Tremayne,"[The War of the Worlds, The War of the Worlds]"
4,Bridgette Wilson,"[Mortal Kombat, Nixon, House on Haunted Hill]"


First we need to convert the .csv that we are given to the format that we want

##### Nodes -> Films Edges -> Films with same actor

In [2]:
df["Movies"][18]

['Faces', 'Faces']

We have duplicated movies in some actors

In [3]:
for i in range(df["Movies"].count()):
    df["Movies"][i] = list(set(df["Movies"][i])) #Can't use pandas.unique() cos type of ["Movies"][i] is -> list   
df["Movies"][18]

['Faces']

Now we make the nodes list just by appending all the films

In [4]:
movies = df.iloc[:,1].copy() #We copy the actors series
#movies = movies.to_frame() #Convert it to a dataframe
nodes = pd.DataFrame(columns=['Id','Movie'])
id = 0
for i in range(len(movies)):
    for j in range(len(movies[i])):
        nodes = nodes.append({"Id":id,"Movie":movies[i][j]}, ignore_index=True)
        id+=1
        
print(len(nodes))
nodes.head()

10071


,Id,Movie
0,0,Starman
1,1,National Lampoon's Animal House
2,2,Malcolm X
3,3,Basic
4,4,House on Haunted Hill


In [6]:
#We eliminate the duplicate movies
nodes_list = nodes
nodes_list = nodes_list["Movie"].unique()
nodes_list = pd.DataFrame({'Movie': nodes_list[:]})
nodes_list.rename(columns={0:"Id"})
nodes_list.index.name = "Id"
nodes_list.head()
nodes_list.to_csv("nodes_list.csv")
nodes_list.tail()

,Movie
Id,
1891,Omagh
1892,The Great St Trinian's Train Robbery
1893,Hullabaloo
1894,Wasabi
1895,Hawks


In [7]:
edges = pd.DataFrame(columns=["Source","Target","Weight","Type"])

#is_empty = lambda x,y: edges.loc[(edges["Source"] == x) & (edges["Target"] == y)].count().all() == 0
is_empty = lambda x,y,edges: edges.loc[(edges["Source"] == x) & (edges["Target"] == y)].count()[0] == 0

get_index = lambda movie_name : nodes_list.loc[nodes_list["Movie"] == movie_name, "Movie"].index[0]

def add_weight(x,y,edges):
    edges.loc[(edges["Source"] == x) & (edges["Target"] == y),"Weight"] += 1
    
def get_edges(edges):
    for i in range(df["Movies"].count()):
        for j in range(len(df["Movies"][i])):
        #Dont go over the full list -> quicker
        #why? -> last element connected in previous iter
            for k in range(j+1,len(df["Movies"][i])): #Dont add equal edges or already added
                source = get_index(df["Movies"][i][j])
                target = get_index(df["Movies"][i][k])
                if(source != target):
                    if(not is_empty(source,target,edges)):    #if there is one in k->j, we increase weight
                        add_weight(source,target,edges)
                    elif(not is_empty(target,source,edges)):  #if there is one in j->k, we increase weight
                        add_weight(target,source, edges)
                    else:
                        #both are empty -> add another edge
                        edges = edges.append({"Source":source,"Target":target,"Weight":1,"Type":"Undirected"}, ignore_index=True)

    return edges
edges = get_edges(edges)
len(edges)
edges.tail()

,Source,Target,Weight,Type
18792,83,1267,1,Undirected
18793,83,1272,1,Undirected
18794,906,1272,1,Undirected
18795,1267,1272,1,Undirected
18796,644,82,1,Undirected


In [8]:
edges.rename(columns={0:"Id"}) #Adding an index column
edges.index.name = "Id"
edges_test = edges.copy()

In [9]:
edges_test.drop_duplicates(subset=["Source","Target"], inplace=True)
print(len(edges))
len(edges_test)

18797


18797

In [10]:
edges_test.to_csv("edges_list.csv")